### Import necessary modules

In [1]:
import requests
from bs4 import BeautifulSoup as bs

import json # library to handle JSON files

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import geocoder

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

import warnings
warnings.filterwarnings("ignore")

### use the requests module to get the html content at the wiki page and store the results in the variable 'html'

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url)
html.text[:100]

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title'

### parse the html using BeautifulSoup 

In [3]:
soup = bs(html.content,'html.parser')
soup.text[:100]

'\n\n\n\nList of postal codes of Canada: M - Wikipedia\ndocument.documentElement.className = document.docu'

### Extract the table information from the Beautiful soup object

In [4]:
table = soup.find('table')
table.text[:100]

'\n\nPostcode\nBorough\nNeighbourhood\n\n\nM1A\nNot assigned\nNot assigned\n\n\nM2A\nNot assigned\nNot assigned\n\n\nM'

### create a dataframe from the table variable

In [5]:
df = pd.read_html(str(table))[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


### grab the first row of the dataframe in order to set them as the columns

In [6]:
cols = df.loc[0]
cols

0         Postcode
1          Borough
2    Neighbourhood
Name: 0, dtype: object

### update the dataframe by setting the correct values for the column names

In [7]:
df2 = df.loc[1:]
df2.columns = cols
df2.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


### Only grab those rows that have a Borough assigned

In [8]:
df3 = df2[df2['Borough'] != 'Not assigned']
df3.head()

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


### Check to see how many Neighbourhood entries with a value of 'Not assigned' can be assigned the corresponding 'Borough' value.  There is only one: Queen's Park.

In [9]:
df3['Borough'][df3.Neighbourhood == 'Not assigned'].value_counts()

Queen's Park    1
Name: Borough, dtype: int64

### Check the remaining unassigned Neighbourhood

In [10]:
df3[df3['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood
9,M7A,Queen's Park,Not assigned


### Set the Neighbourhood "Not Assigned" value to corresponding "Borough" value

In [11]:
mask = df3['Neighbourhood'] == 'Not assigned'
df3.loc[mask,'Neighbourhood'] = df3.loc[mask,'Borough']

### Confirm that the change took effect

In [12]:
df3[df3['Neighbourhood'] == 'Not assigned']

,Postcode,Borough,Neighbourhood


### Group the Neighbourhoods by Postcode and Borough
- apply the "join" method, using ", " to the Neighbourhood column in each group to create a single string of comma separated values of all the neighbourhoods in that the group.

In [13]:
df_groups = df3.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ", ".join(x)).to_frame().reset_index()
df_groups

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Print the shape of the Dataframe to show the number of rows and columns

In [14]:
df_groups.shape

(103, 3)

### Create a new dataframe that is a copy of the DF from part 1 to perform the operations required for part 2

In [15]:
df4 = df_groups.copy()

### Get the latitude and longitude
- attempt to use the geocoder module

In [16]:
import geocoder

#### use the first postal code to test

In [17]:
postal_code = df4.loc[0,'Postcode']
postal_code

'M1B'

#### get the lat and long for the M1B postal code

In [18]:
g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
lat_lng_coords = g.latlng

#### assign the lat and long values to their repsective variables
- Note: the geocoder module is not working on this single example

In [19]:
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

TypeError: 'NoneType' object is not subscriptable

### Try 10 attempts to get the geocoder module to work
- note: the geocoder module fails after 10 attempts

In [20]:
geocoder.google('{}, Toronto, Ontario'.format(postal_code))

<[OVER_QUERY_LIMIT] Google - Geocode [empty]>

In [21]:
postal_code = 'M1V'
postal_code

'M1V'

In [22]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
counter = 1
while(counter < 11):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    print(counter)
    counter+=1

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print(counter)
print('Latitude {}'.format(latitude))
print('Longitude {}'.format(longitude))

1
2
3
4
5
6
7
8
9
10


TypeError: 'NoneType' object is not subscriptable

### Conclusion: the geocoder module is too unreliable, therefore use the supplied CSV of latitude and longitude values for each Postal Code

### Load the latitude and longitude data stored in the supplied csv

In [23]:
df_coordinates = pd.read_csv('../notebooks/data/Geospatial_Coordinates.csv')
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### rename the CSV file column name "Postal Code" to match the "Postcode" column name from the wikipage

In [24]:
df_coordinates.rename(columns={'Postal Code':df4.columns[0]},inplace=True)
df_coordinates.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge the latitude and longitude dataframes on the "Postcode" column

In [25]:
df5 = df4.merge(df_coordinates,on='Postcode')
df5

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Check for Null values to see if merge did not work anywhere

In [26]:
df5.isnull().sum()

Postcode         0
Borough          0
Neighbourhood    0
Latitude         0
Longitude        0
dtype: int64

### Explore and cluster the neighborhoods in Toronto
- start by finding only those 

In [83]:
toronto_idx = df5.Borough.str.contains("Toronto")
toronto_df = df5.loc[toronto_idx,:].reset_index(drop=True)

In [84]:
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### note the number of total postcodes to explore the neighbourhoods in Toronto

In [85]:
toronto_df.Borough.value_counts().sum()

38

### load foursquare credentials for use with API to extra neighbourhood information

In [73]:
CLIENT_ID = 'abc' # your Foursquare ID -> removed for the grading portion
CLIENT_SECRET = 'xyz' # your Foursquare Secret -> removed for the grading portion
VERSION = '20180604'
#LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 42R2YOP4MTNEEGBLCFEBVLKE1S1EFC5AUWUIS45M3Q4Y00SK
CLIENT_SECRET:X4THSP32AF2EVUC4M4O2IP3KYLKSF2WHW5BNKSQM4GYBF2N1


### function from the lab that gets the venue details for the 

In [114]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhoods', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### use the function above to get the venue information

In [156]:
import time
start = time.time()
toronto_venues = getNearbyVenues(toronto_df.Postcode, toronto_df.Latitude , toronto_df.Longitude)
print("It took {:.2f} seconds to complete this request".format(time.time()-start))

M4E
M4K
M4L
M4M
M4N
M4P
M4R
M4S
M4T
M4V
M4W
M4X
M4Y
M5A
M5B
M5C
M5E
M5G
M5H
M5J
M5K
M5L
M5N
M5P
M5R
M5S
M5T
M5V
M5W
M5X
M6G
M6H
M6J
M6K
M6P
M6R
M6S
M7Y
It took 41.31 seconds to complete this request


### Print the shape of the dataframe to see how many venues were returned and the head to verify first few rows

In [157]:
print(toronto_venues.shape)
toronto_venues.head()

(1699, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
1,M4E,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,M4E,43.676357,-79.293031,St-Denis Studios Inc.,43.675031,-79.288022,Music Venue
3,M4E,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [127]:
# Look at the number of venues by the neighbourhood
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
M4E      4
M4K     42
M4L     21
M4M     40
M4N      4
M4P      8
M4R     19
M4S     36
M4T      3
M4V     14
M4W      4
M4X     46
M4Y     88
M5A     49
M5B    100
M5C    100
M5E     54
M5G     82
M5H    100
M5J    100
M5K    100
M5L    100
M5N      2
M5P      4
M5R     23
M5S     37
M5T    100
M5V     14
M5W     92
M5X    100
M6G     16
M6H     17
M6J     65
M6K     21
M6P     24
M6R     15
M6S     37
M7Y     18
Name: Venue, dtype: int64

### Get the number of unique categories returned

In [159]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


### Look at the top 10 category types

In [160]:

toronto_venues['Venue Category'].value_counts()[:10]

Coffee Shop            159
Café                    97
Restaurant              49
Hotel                   43
Italian Restaurant      43
Bar                     40
Bakery                  36
Pizza Place             31
Park                    31
Japanese Restaurant     30
Name: Venue Category, dtype: int64

## 3. Analyze Each Neighborhood
- one hot encode categories and store results in a new data frame called toronto_onehot

In [229]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postcode'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1699, 233)


,Postcode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

####  group rows by neighborhood postcode and get the mean of the frequency of occurrence of each category

In [232]:
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped.head(10)

,Postcode,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000,0.000000,0.00000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.00000,0.000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.250000,0.0,0.0,0.0,0.000,0.0,0.0,0.000000,0.00000,0.000,0.0,0.0,0.0,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.

In [233]:
# check resulting shape to ensure that all 38 postcodes are represented
toronto_grouped.shape

(38, 233)

#### print the top 5 most common venues for each neighbourhood postcode

In [235]:
num_top_venues = 5

for hood in toronto_grouped['Postcode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4E----
               venue  freq
0        Coffee Shop  0.25
1                Pub  0.25
2        Music Venue  0.25
3       Neighborhood  0.25
4  Accessories Store  0.00


----M4K----
                venue  freq
0    Greek Restaurant  0.21
1         Coffee Shop  0.10
2      Ice Cream Shop  0.07
3           Bookstore  0.05
4  Italian Restaurant  0.05


----M4L----
                  venue  freq
0        Sandwich Place  0.10
1           Pizza Place  0.05
2                   Pub  0.05
3  Fast Food Restaurant  0.05
4          Burger Joint  0.05


----M4M----
                venue  freq
0                Café  0.10
1         Coffee Shop  0.08
2  Italian Restaurant  0.05
3           Gastropub  0.05
4              Bakery  0.05


----M4N----
                venue  freq
0                Park  0.25
1  Dim Sum Restaurant  0.25
2         Swim School  0.25
3            Bus Line  0.25
4   Accessories Store  0.00


----M4P----
               venue  freq
0              Hotel  0.12
1  Food & Drink Sh

### Place top 10 most common venue types by mean score of count into a dataframe

In [236]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [237]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postcode'] = toronto_grouped['Postcode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Neighborhood,Coffee Shop,Pub,Music Venue,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M4K,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Spa,Indian Restaurant,Diner,Bakery
2,M4L,Sandwich Place,Pet Store,Hotel,Brewery,Liquor Store,Burger Joint,Fast Food Restaurant,Burrito Place,Fish & Chips Shop,Italian Restaurant
3,M4M,Café,Coffee Shop,Bakery,Italian Restaurant,Gastropub,American Restaurant,Fish Market,Juice Bar,Latin American Restaurant,New American Restaurant
4,M4N,Bus Line,Park,Swim School,Dim Sum Restaurant,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
5,M4P,Breakfast Spot,Clothing Store,Sandwich Place,Food & Drink Shop,Dance Studio,Hotel,Burger Joint,Park,Eastern European Restaurant,Dog Run
6,M4R,Coffee Shop,Sporting Goods Shop,Clothing Store,Health & Beauty Service,Fast Food Restaurant,Metro Station,Mexican Restaurant,Diner,Dessert Shop,Park
7,M4S,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Coffee Shop,Italian Restaurant,Café,Pharmacy,Seafood Restaurant,Toy / Game Store
8,M4T,Playground,Park,Intersection,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
9,M4V,Coffee Shop,Pub,American Restaurant,Supermarket,Vietnamese Restaurant,Light Rail Station,Convenience Store,Pizza Place,Sushi Restaurant,Sports Bar


## 4. Cluster Neighborhoods
- Run *k*-means to cluster the neighborhood into 5 clusters.

In [240]:
# set number of clusters
kclusters = 5

# remove the "Neighbourhood" column from the toronto_grouped dataframe
toronto_grouped_clustering = toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 2, 3, 3, 3, 0, 3], dtype=int32)

In [241]:
pd.Series(kmeans.labels_).value_counts()

3    33
0     2
4     1
2     1
1     1
dtype: int64

In [242]:
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,3
3,M4M,East Toronto,Studio District,43.659526,-79.340923,3
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2


In [243]:
toronto_merged = toronto_df

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,3
3,M4M,East Toronto,Studio District,43.659526,-79.340923,3
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2


In [244]:
# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postcode'), on='Postcode')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Neighborhood,Coffee Shop,Pub,Music Venue,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Spa,Indian Restaurant,Diner,Bakery
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,3,Sandwich Place,Pet Store,Hotel,Brewery,Liquor Store,Burger Joint,Fast Food Restaurant,Burrito Place,Fish & Chips Shop,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,3,Café,Coffee Shop,Bakery,Italian Restaurant,Gastropub,American Restaurant,Fish Market,Juice Bar,Latin American Restaurant,New American Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Bus Line,Park,Swim School,Dim Sum Restaurant,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


Finally, let's visualize the resulting clusters

In [315]:
# create map of Manhattan
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
print(ys)
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters

[array([0, 1, 2, 3, 4]), array([ 1,  3,  7, 13, 21]), array([ 2,  7, 20, 41, 70]), array([  3,  13,  41,  87, 151]), array([  4,  21,  70, 151, 264])]


## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [259]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,0,Playground,Park,Intersection,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
10,Downtown Toronto,0,Park,Playground,Trail,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [314]:
cluster1_cats = pd.Series(cluster1[cluster1.columns[2:4]].stack().unique()).values.tolist()
cluster1_cats

['Playground', 'Park']

### Name for Cluster1: The Parks

#### Cluster 2

In [283]:
cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster2

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Garden,Pool,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [313]:
cluster2_cats = pd.Series(cluster2[cluster2.columns[2:4]].stack().unique()).values.tolist()
cluster2_cats

['Garden', 'Pool']

### Name for Cluster2: The Gardens

#### Cluster 3

In [285]:
cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster3

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Bus Line,Park,Swim School,Dim Sum Restaurant,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [308]:
cluster3_cats = pd.Series(cluster3[cluster3.columns[2:]].stack().unique()).values.tolist()
cluster3_cats

['Bus Line',
 'Park',
 'Swim School',
 'Dim Sum Restaurant',
 'Diner',
 'Farmers Market',
 'Falafel Restaurant',
 'Event Space',
 'Ethiopian Restaurant',
 'Electronics Store']

### Cluster 3 Name: Public Transit District

#### Cluster 4

In [287]:
cluster4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,3,Neighborhood,Coffee Shop,Pub,Music Venue,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,East Toronto,3,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Spa,Indian Restaurant,Diner,Bakery
2,East Toronto,3,Sandwich Place,Pet Store,Hotel,Brewery,Liquor Store,Burger Joint,Fast Food Restaurant,Burrito Place,Fish & Chips Shop,Italian Restaurant
3,East Toronto,3,Café,Coffee Shop,Bakery,Italian Restaurant,Gastropub,American Restaurant,Fish Market,Juice Bar,Latin American Restaurant,New American Restaurant
5,Central Toronto,3,Breakfast Spot,Clothing Store,Sandwich Place,Food & Drink Shop,Dance Studio,Hotel,Burger Joint,Park,Eastern European Restaurant,Dog Run
6,Central Toronto,3,Coffee Shop,Sporting Goods Shop,Clothing Store,Health & Beauty Service,Fast Food Restaurant,Metro Station,Mexican Restaurant,Diner,Dessert Shop,Park
7,Central Toronto,3,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Coffee Shop,Italian Restaurant,Café,Pharmacy,Seafood Restaurant,Toy / Game Store
9,Central Toronto,3,Coffee Shop,Pub,American Restaurant,Supermarket,Vietnamese Restaurant,Light Rail Station,Convenience Store,Pizza Place,Sushi Restaurant,Sports Bar
11,Downtown Toronto,3,Coffee Shop,Restaurant,Café,Bakery,Pizza Place,Italian Restaurant,Pub,Park,Indian Restaurant,Butcher
12,Downtown Toronto,3,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Café,Gastropub,Fast Food Restaurant,Bubble Tea Shop


In [312]:
cluster4_cats = pd.Series(cluster4[cluster4.columns[2:4]].stack().unique()).values.tolist()
print(cluster4_cats)

['Neighborhood', 'Coffee Shop', 'Greek Restaurant', 'Sandwich Place', 'Pet Store', 'Café', 'Breakfast Spot', 'Clothing Store', 'Sporting Goods Shop', 'Pizza Place', 'Pub', 'Restaurant', 'Japanese Restaurant', 'Park', 'Cocktail Bar', 'Hotel', 'Yoga Studio', 'Vegetarian / Vegan Restaurant', 'Airport Lounge', 'Airport Service', 'Grocery Store', 'Supermarket', 'Bakery', 'Bar', 'Mexican Restaurant', 'Gift Shop', 'Light Rail Station']


### Cluster 4 Name: Coffee House

In [293]:
cluster5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
cluster5

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,4,Trail,Mexican Restaurant,Jewelry Store,Sushi Restaurant,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [294]:
cluster5_cats = pd.Series(cluster5[cluster5.columns[2:7]].stack().unique()).values.tolist()
cluster5_cats

['Trail',
 'Mexican Restaurant',
 'Jewelry Store',
 'Sushi Restaurant',
 'Yoga Studio']

### Cluster 5 Name: The Trails